In [1]:
import json
import requests
import pandas as pd

In [2]:
# startpos2와 endpos2를 붙여 고유한 것을 가져와 df_new로 만듭니다.
 
df = pd.read_csv("master_cleansing_map_after.csv", index_col = 0)
dummy = df[['startpos2', 'endpos2', 'slat', 'slng', 'elat', 'elng']]
dummy.columns = ['start', 'end', 'startlat', 'startlng', 'endlat', 'endlng']

,start,end,startlat,startlng,endlat,endlng,direction
0,장위제1동,회기동,127.043713,37.614125,127.055313,37.590808,장위제1동회기동
1,면목제7동,상계1동,127.086985,37.578995,127.054978,37.679901,면목제7동상계1동
2,정릉제3동,수유제3동,127.004186,37.608958,127.023125,37.638715,정릉제3동수유제3동
3,고척제2동,은천동,126.858712,37.506706,126.942428,37.485309,고척제2동은천동
4,회기동,장위제2동,127.055313,37.590808,127.054521,37.612390,회기동장위제2동
...,...,...,...,...,...,...,...
32513,여의동,서초2동,126.934584,37.517723,127.024960,37.492080,여의동서초2동
32514,쌍문제4동,아현동,127.028425,37.656482,126.952150,37.547125,쌍문제4동아현동
32515,상암동,방화제2동,126.894688,37.578325,126.806755,37.566700,상암동방화제2동
32516,반포1동,방배2동,127.013507,37.505089,126.985511,37.479744,반포1동방배2동


In [ ]:
# startpos2와 endpos2를 붙여 고유한 것을 가져와 df_new로 만듭니다.
a = []
for i in range(len(dummy)):
    a.append(dummy['start'][i] + dummy['end'][i])
dummy['direction'] = a
df_new = dummy.drop_duplicates()
df_new = df_new.reset_index(drop = True)
df_new

In [3]:
# direction 5 와 direction 15 API 키를 각각 생성합니다. 각 api는 하루 6000건이 최대입니다.

# 총 4명이므로, 8개의 api를 생성할 수 있습니다. df_new의 개수는 32518개이므로, df_new를 4100개씩 나눠 진행합니다.
# 각 데이터프레임을 각각 api 돌린 후, 다 뽑아내오면 하나로 합쳐줍니다.

# 하나로 합쳐진 데이터프레임 df_new는 direction의 고유값이기 때문에, 이후 df에 join merge 시키면 됩니다.

df_new1 = df_new[:4100] # 규동(direction 5 사용)
df_new2 = df_new[4100:8200] # 규동(direction 15 사용)
df_new3 = df_new[8200:12300] # 은지(direction 5 사용)
df_new4 = df_new[12300:16400] # 은지(direction 15 사용)
df_new5 = df_new[16400:20500] # 신석(direction 5 사용)
df_new6 = df_new[20500:24600] # 신석(direction 15 사용)
df_new7 = df_new[24600:28700] # 민재(direction 5 사용)
df_new8 = df_new[28700:] # 민재(direction 15 사용)

len(df_new1), len(df_new2), len(df_new3), len(df_new4), len(df_new5), len(df_new6), len(df_new7), len(df_new8)

(4100, 4100, 4100, 4100, 4100, 4100, 4100, 3818)

# 여기서부터 조심히 사용해주세요. api를 사용합니다.

# Direction 5 API와 Direction 15 API 수정하세요.

In [4]:
# direction 5 API에 대한 ID, Key 입력
dir_5_ID = "guu4qvwey0"
dir_5_Key = "T3hmEFpld9J7XLZHA3OJRHjzlWWIrbbmf3JEcZjz"

# direction 15 API에 대한 ID, Key 입력
dir_15_ID = "kqos0zymqt"
dir_15_Key = "zQevQYQJs8Ddbk7pTqGV7eOsrnYhc21qGwea1lMT"

In [5]:
## 여기는 수정하지 마세요!

# 헤더 설정
headers_5 = {'X-NCP-APIGW-API-KEY-ID' : dir_5_ID, 'X-NCP-APIGW-API-KEY' : dir_5_Key}
headers_15 = {'X-NCP-APIGW-API-KEY-ID' : dir_15_ID, 'X-NCP-APIGW-API-KEY' : dir_15_Key}

In [12]:
## 이거 지워주세요!!!!!!!!!!

distance = []
duration = []
taxiFare = []
for i in range(len(df_new1)):
    slat = df_new1['startlat'][i]
    slng = df_new1['startlng'][i]
    elat = df_new1['endlat'][i]
    elng = df_new1['endlng'][i]
    url = f"https://naveropenapi.apigw.ntruss.com/map-direction/v1/driving?start={slat},{slng}&goal={elat},{elng}"
    try:
        document = requests.get(url, headers=headers_5).json()
        distance.append(document['route']['traoptimal'][0]['summary']['distance'])
        duration.append(document['route']['traoptimal'][0]['summary']['duration'] / 1000)
        taxiFare.append(document['route']['traoptimal'][0]['summary']['taxiFare'])
    except:
        distance.append(0)
        duration.append(0)
        taxiFare.append(0)

In [6]:
# 기타 설정
distance = []
duration = []
taxiFare = []

"""
여기부터는 df_new 자기 숫자에 맞춰 진행하세요.
"""
# 주의! 여기서부터 api를 사용합니다. headers_5 먼저 돌립니다.
# for > range > len 안의 값과 url 내의 변수를 꼭 변경해주세요.
for i in range(len(df_new1)):
    slat = df_new1['startlat'][i]
    slng = df_new1['startlng'][i]
    elat = df_new1['endlat'][i]
    elng = df_new1['endlng'][i]
    url = f"https://naveropenapi.apigw.ntruss.com/map-direction/v1/driving?start={slat},{slng}&goal={elat},{elng}"
    try:
        document = requests.get(url, headers=headers_5).json()
        distance.append(document['route']['traoptimal'][0]['summary']['distance'])
        duration.append(document['route']['traoptimal'][0]['summary']['duration'] / 1000)
        taxiFare.append(document['route']['traoptimal'][0]['summary']['taxiFare'])
    except:
        distance.append(0)
        duration.append(0)
        taxiFare.append(0)

# df_new에 각 변수를 추가하여 넣습니다.
df_new1['distance'] = distance
df_new1['duration'] = duration
df_new1['taxiFare'] = taxiFare

# csv로 저장합니다.
df_new1.to_csv('df_new1.csv', encoding='utf-8')

KeyError: 'route'

In [ ]:
# 기타 설정
distance = []
duration = []
taxiFare = []

"""
여기부터는 df_new 자기 숫자에 맞춰 진행하세요.
"""
# 주의! api를 사용합니다. headers_15 돌립니다.
# for > range > len 안의 값과 url 내의 변수를 꼭 변경해주세요. 
for i in range(len(df_new2)):
    slat = df_new2['startlat'][i]
    slng = df_new2['startlng'][i]
    elat = df_new2['endlat'][i]
    elng = df_new2['endlng'][i]
    url = f"https://naveropenapi.apigw.ntruss.com/map-direction-15/v1/driving?start={slat},{slng}&goal={elat},{elng}"
    try:
        document = requests.get(url, headers=headers_15).json()
        distance.append(document['route']['traoptimal'][0]['summary']['distance'])
        duration.append(document['route']['traoptimal'][0]['summary']['duration'] / 1000)
        taxiFare.append(document['route']['traoptimal'][0]['summary']['taxiFare'])
    except:
        distance.append(0)
        duration.append(0)
        taxiFare.append(0)

# df_new에 각 변수를 추가하여 넣습니다.
# df_new 숫자만 변경해서 쓰세요
df_new2['distance'] = distance
df_new2['duration'] = duration
df_new2['taxiFare'] = taxiFare

# csv로 저장합니다.
df_new2.to_csv('df_new2.csv', encoding='utf-8')